- more coherent context

In [1]:
import os
from OPENAI_KEY import OPENAI_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_KEY

In [2]:
# Prepare the documentr
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

# Merge into a single large document for better RAG performance
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [3]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2048, 512, 128])

In [4]:
nodes = node_parser.get_nodes_from_documents([document])


In [5]:
from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[27].text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.


In [6]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [7]:
from llama_index.core import ServiceContext

# 1. Use node parser to break/window the sentences
# 2. Embed the sentences
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    # embed_model="local:BAAI/bge-large-en-v1.5",
    node_parser=node_parser
)

C:\Users\wongs\AppData\Local\Temp\ipykernel_40680\4059913470.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


- embeddings will be based on that sentence, the parent nodes and other information are retrieve only when it is reranked

In [8]:
nodes[0]

TextNode(id_='066fa5c7-5d96-4cd6-976f-deba8b00583e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='32bd67d9-ce71-4196-9b23-263a8a0d712f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='fdc7ce9a017ce62d4e0b30cc419051a9bb56618a8569f51a2b64f8535fe6244e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e775f409-41b8-422e-84fd-cc2fa7c00129', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='601594bb882270ad63b4547797595e35b471287cec74dcc07134629cbb7d71f5'), <NodeRelationship.CHILD: '5'>: [RelatedNodeInfo(node_id='3b7f423c-7e93-4755-8400-b134c046c1e2', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2b4de016a7c855ab9278e03c30a440bb34e81a1ea68fd3225250abf084e52330'), RelatedNodeInfo(node_id='e2278f02-faa8-4a31-b606-3b644c8df394', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='0c8c4015b5079720c9a99a811c9a8888c9f8ee3b9265eec5507fee681025f69c'), R

In [9]:
from llama_index.core import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)  # all the nodes - parents & leaf

# Only leaf nodes is used for indexing where embeddings are calculated, then the parent nodes can be found in storage_context
# this is just the index class
automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=sentence_context
)

In [10]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# 1. Processor to decide retriving the parent or not
automerging_retriever = automerging_index.as_retriever(similarity_top_k=12)
retriever = AutoMergingRetriever(
    automerging_retriever,
    automerging_index.storage_context,
    verbose=True
)

# 2. Reranked the processed nodes
rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")
auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [11]:
auto_merging_response = auto_merging_engine.query('What is the importance of networking in AI?')

In [12]:
from llama_index.core.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking in AI is crucial as it allows individuals to build a strong professional community that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive guidance, referrals to potential employers, and access to new ideas and perspectives. Additionally, networking helps individuals establish themselves within the AI community, gain recognition for their expertise, and continue to develop their skills through interactions with mentors and peers.

In [13]:
from automerging_utils import get_automerging_query_engine, build_automerging_index

index = build_automerging_index([document],
                                llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
                                save_dir="./merging_index")
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## Evaluation

In [14]:
### 2 layers

In [15]:
auto_merging_index_0 = build_automerging_index(documents,
                                               llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
                                               save_dir="./merging_index_0",
                                               chunk_sizes=[2048, 512])
auto_merging_engine_0 = get_automerging_query_engine(auto_merging_index_0, 
                                                     similarity_top_k=12,
                                                     rerank_top_n=6)

In [16]:
# Set up trurecorder
from utils import get_prebuilt_trulens_recorder
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

tru_recorder_0 = get_prebuilt_trulens_recorder(auto_merging_engine_0, app_id="2_layers")

✅ In Answer relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wongs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [17]:
# Import the eval questions
eval_questions = []
with open('./generated_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)

In [18]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [19]:
run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder_0, query_engine=auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: fe9c5eae-e278-4ebf-b8c0-d5c833efcdce.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: 1bd98ca7-6442-4136-9773-64913ab9b72c.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: c900ce78-5a72-49f9-90bb-662ef01f6ffb.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: ed834e9d-b0ec-4584-9117-ddb6b23fe8ab.
> Parent node text: PAGE 23Each project is only one step on a longer journey, hopefully one that has a positive impac...

> Merging 1 nodes into parent node.
> Parent node id: b4f07b1b-5ce7-40c5-9530-72a800ad4852.
> Parent node text: PAGE 29If you’re preparing to sw

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

> Merging 1 nodes into parent node.
> Parent node id: 9f5ed503-7737-43e4-837c-65090fac6131.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...

> Merging 2 nodes into parent node.
> Parent node id: 006d60bd-96a0-49dd-aafc-c0cbc0496e18.
> Parent node text: PAGE 20Working on projects requires making tough choices about what to build and how to go 
about...

> Merging 1 nodes into parent node.
> Parent node id: 920582e3-3c21-4694-8f6b-65c02d41872f.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: c900ce78-5a72-49f9-90bb-662ef01f6ffb.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: aaafc4aa-cfc9-4b78-8459-5699899034c0.
> Parent node text: PAGE 36Keys to Building a Career

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/6 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

> Merging 1 nodes into parent node.
> Parent node id: c04d8115-9808-4c2f-9a36-ea1503b37f2a.
> Parent node text: PAGE 6The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building excit...

> Merging 1 nodes into parent node.
> Parent node id: b0a2bcde-4adc-43a6-8101-fa0f293729a1.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 70790397-3e94-480c-8a55-77ab63d9a5d5.
> Parent node text: PAGE 7These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For e...

> Merging 1 nodes into parent node.
> Parent node id: aaafc4aa-cfc9-4b78-8459-5699899034c0.
> Parent node text: PAGE 36Keys to Building a Career in AI CHAPTER 10
Of all the steps in building a career, this 
on...

> Merging 1 nodes into parent node.
> Parent node id: 920582e3-3c21-4694-8f6b-65c02d41872f.
> Parent node text: PAGE 18It goes without saying th

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
auto_merging_index_1= build_automerging_index(documents,
                                               llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
                                               save_dir="./merging_index_1",
                                               chunk_sizes=[2048, 512, 128])
auto_merging_engine_1 = get_automerging_query_engine(auto_merging_index_1, 
                                                     similarity_top_k=12,
                                                     rerank_top_n=6)
tru_recorder = get_prebuilt_trulens_recorder(auto_merging_engine_1, app_id="3_layers")


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder, query_engine=auto_merging_engine_1)


> Merging 3 nodes into parent node.
> Parent node id: 3174583f-eefe-4c44-a2cb-a3b183ccf7cf.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 5 nodes into parent node.
> Parent node id: 3d36dbd1-d247-4813-aeba-d518f2d8790c.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 2bcb49ab-2e4b-4484-823f-656a595832ba.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: ed89706d-536d-492a-8be1-28aa5c9e1504.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/6 [00:00<?, ?it/s]

> Merging 3 nodes into parent node.
> Parent node id: 56fcc644-5c7f-48a5-8221-7d2b281f150f.
> Parent node text: PAGE 6The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building excit...

> Merging 2 nodes into parent node.
> Parent node id: 71daf6c1-b9cc-4a9b-9562-68909ffe786c.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 67fd1d18-4c45-46c6-904b-25fcc5d531a6.
> Parent node text: PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

> Merging 1 nodes into parent node.
> Parent node id: fea5c990-e58e-4546-b162-6eb653a060af.
> Parent node text: PAGE 5Three Steps to 
Career GrowthCHAPTER 1

> Merging 1 nodes into parent node.
> Parent node id: ceaf5d22-4dd4-4399-a9dc-ba1bff09902f.
> Parent node text: PAGE 6The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building excit...

> Merging 1 nodes

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Dashboard started at http://10.91.30.3:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>